# TRAINING NOTEBOOK

## Environment

In [1]:
%reload_ext autoreload
%autoreload 2

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
# Change main system path to be able to run code from src folder
import sys
p = sys.path[0]
# Mac OS
if sys.path[0].endswith('/notebooks'):
    main_path = p[:-len('/notebooks')]
if sys.path[0].endswith('/techdoc/content'):
    main_path = p[:-len('/techdoc/content')]
    
# Windows OS
if sys.path[0].endswith('\\notebooks'): 
    main_path = p[:-len('\\notebooks')]
if sys.path[0].endswith('\\techdoc\content'): 
    main_path = p[:-len('\\techdoc\content')]

sys.path[0] = main_path

In [98]:
import os, gc, itertools
from termcolor import colored
from tqdm import tqdm

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, f1_score
from sklearn.model_selection import cross_val_predict
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
from scipy.signal import find_peaks
from scipy.ndimage.filters import gaussian_filter1d
from sklearn.metrics import auc

from src import (config, describe_data, features,
                 preprocess, training)

# DATA PREPARATION

In [4]:
# ===== LOAD DATA ======
metadata = pd.read_csv(config.DATA_DIR + 'metadata.csv')
print(f'Metadata: {metadata.shape}')

train_labels = pd.read_csv(config.DATA_DIR + 'train_labels.csv')
print(f'Train labels: {train_labels.shape}')

valid_labels = pd.read_csv(config.DATA_DIR + 'val_labels.csv')
print(f'Train labels: {valid_labels.shape}')

submission = pd.read_csv(config.DATA_DIR + 'submission_format.csv')
print(f'Submission: {submission.shape}')

Metadata: (1570, 5)
Train labels: (766, 11)
Train labels: (293, 11)
Submission: (804, 11)


In [5]:
# ===== MODEL SAMPLES ======
train = metadata[metadata.split == 'train'].copy().reset_index(drop=True)
print(f'TRAIN: {train.shape}')

valid = metadata[metadata.split == 'val'].copy().reset_index(drop=True)
print(f'VALID: {valid.shape}')

test = metadata[metadata.split == 'test'].copy().reset_index(drop=True)
print(f'TEST: {test.shape}')

TRAIN: (766, 5)
VALID: (293, 5)
TEST: (511, 5)


In [6]:
# ===== FILE PATHS OF SAMPLES =====
train_files = metadata[metadata.split == 'train']['features_path'].to_dict()
valid_files = metadata[metadata.split == 'val']['features_path'].to_dict()
test_files = metadata[metadata.split == 'test']['features_path'].to_dict()
# Train & Valid files
trva_files = train_files.copy()
trva_files.update(valid_files)
# All files
all_test_files = valid_files.copy()
all_test_files.update(test_files)

# Ion type list
ion_list = list(np.arange(0,100,1.0))
ion_list.remove(4.0)

# Get the names of the target columns in a list
target_labels_list = [i for i in train_labels.columns if i not in ['sample_id']]
print(target_labels_list)

['basalt', 'carbonate', 'chloride', 'iron_oxide', 'oxalate', 'oxychlorine', 'phyllosilicate', 'silicate', 'sulfate', 'sulfide']


# MODELS

In [14]:
# Data frame to save local CV results
models_log_loss = pd.DataFrame(index=target_labels_list)

## Logistic Regression - Benchmark

In [15]:
train_cv_loss_LR, train_full_clf_LR, submission_LR = training.train_tbl(
    df_train=fts_maxrelabund_tempion,
    df_labels=train_labels,
    target_list=target_labels_list,
    df_test=fts_maxrelabund_tempion_VT,
    model_algo='LR_reg',
    sub_name='LR_reg'
    )
models_log_loss['LR_reg'] = models_log_loss.index.map(train_cv_loss_LR)


Average Log Loss: 0.2759
Log Loss per Label:
{'basalt': 0.27678260590883075, 'carbonate': 0.2612545003933846, 'chloride': 0.2842979960325033, 'iron_oxide': 0.3824699336087238, 'oxalate': 0.004059239489520996, 'oxychlorine': 0.2991500982787292, 'phyllosilicate': 0.4074758334090647, 'silicate': 0.35168474392654814, 'sulfate': 0.34471883287139204, 'sulfide': 0.14713584783404984}


In [16]:
submission_LR.head()

,basalt,carbonate,chloride,iron_oxide,oxalate,oxychlorine,phyllosilicate,silicate,sulfate,sulfide
sample_id,,,,,,,,,,
S0766,1.868003e-07,0.000062,0.000004,1.125625e-07,0.000052,0.000090,8.296115e-08,0.998771,0.000002,0.000002
S0767,2.259753e-01,0.261281,0.007587,3.353358e-01,0.000132,0.226904,1.458527e-02,0.879255,0.057940,0.009740
S0768,5.743717e-01,0.508643,0.000804,2.181478e-01,0.000763,0.277165,8.997608e-01,0.685684,0.249768,0.002388
S0769,1.895572e-02,0.092020,0.042351,7.053178e-02,0.000083,0.977360,1.079748e-01,0.086535,0.370443,0.003093
S0770,8.417039e-04,0.008442,0.048491,3.099149e-01,0.000358,0.862539,5.699419e-01,0.003204,0.001448,0.019214


## XGB

- numerical data needs to be scaled
- categorical data needs to be encoded

In [20]:
train_cv_loss_XGB, train_full_clf_XGB, submission_XGB = training.train_tbl(
    df_train=fts_maxrelabund_tempion,
    df_labels=train_labels,
    target_list=target_labels_list,
    df_test=fts_maxrelabund_tempion_VT,
    model_algo='XGB',
    sub_name='XGB'
    )
models_log_loss['XGB'] = models_log_loss.index.map(train_cv_loss_XGB)

/Users/itacdonev/opt/miniconda3/envs/nasamars/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Users/itacdonev/opt/miniconda3/envs/nasamars/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Users/itacdonev/opt/miniconda3/envs/nasamars/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Users/itacdonev/opt/miniconda3/envs/nasamars/lib/python3.9/s


Average Log Loss: 0.1934
Log Loss per Label:
{'basalt': 0.22565078102787242, 'carbonate': 0.13225618815649615, 'chloride': 0.22720553328986837, 'iron_oxide': 0.254578396456081, 'oxalate': 0.027906252811262605, 'oxychlorine': 0.20850854126821824, 'phyllosilicate': 0.29730870486992084, 'silicate': 0.2575651139000811, 'sulfate': 0.2245971262183839, 'sulfide': 0.07831794841140914}


## XGB - optimized

- numerical data needs to be scaled
- categorical data needs to be encoded

In [17]:
train_cv_loss_XGB_opt, train_full_clf_XGB_opt, submission_XGB_opt = training.train_tbl(
    df_train=fts_maxrelabund_tempion,
    df_labels=train_labels,
    target_list=target_labels_list,
    df_test=fts_maxrelabund_tempion_VT,
    model_algo='XGB_opt',
    sub_name='XGB_opt'
    )
models_log_loss['XGB_opt'] = models_log_loss.index.map(train_cv_loss_XGB_opt)

/Users/itacdonev/opt/miniconda3/envs/nasamars/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Users/itacdonev/opt/miniconda3/envs/nasamars/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Users/itacdonev/opt/miniconda3/envs/nasamars/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Users/itacdonev/opt/miniconda3/envs/nasamars/lib/python3.9/s


Average Log Loss: 0.1746
Log Loss per Label:
{'basalt': 0.19665137401195665, 'carbonate': 0.12102018407547, 'chloride': 0.2043376334408594, 'iron_oxide': 0.22611550844240597, 'oxalate': 0.029272685140647313, 'oxychlorine': 0.19420209851804687, 'phyllosilicate': 0.25951556825253963, 'silicate': 0.23317893970805786, 'sulfate': 0.20317408057798922, 'sulfide': 0.07880694186687044}


In [18]:
submission_XGB_opt.head()

,basalt,carbonate,chloride,iron_oxide,oxalate,oxychlorine,phyllosilicate,silicate,sulfate,sulfide
sample_id,,,,,,,,,,
S0766,0.002264,0.002201,0.003759,0.004286,0.001020,0.004258,0.008280,0.482346,0.005675,0.001228
S0767,0.099640,0.044188,0.001508,0.041713,0.000688,0.008322,0.019387,0.208535,0.017686,0.000710
S0768,0.342900,0.187369,0.001210,0.004061,0.001485,0.002831,0.251529,0.916730,0.016389,0.000994
S0769,0.001961,0.002187,0.019743,0.035369,0.000688,0.965160,0.030881,0.002617,0.932003,0.002146
S0770,0.003582,0.002337,0.002956,0.171626,0.001548,0.994668,0.959515,0.001145,0.003278,0.000646


## SVC

In [25]:
train_cv_loss_SVC, train_full_clf_SVC, submission_SVC = training.train_tbl(
    df_train=fts_maxrelabund_tempion,
    df_labels=train_labels,
    target_list=target_labels_list,
    df_test=fts_maxrelabund_tempion_VT,
    model_algo='SVC',
    sub_name='SVC'
    )
models_log_loss['SVC'] = models_log_loss.index.map(train_cv_loss_SVC)


Average Log Loss: 0.2401
Log Loss per Label:
{'basalt': 0.25070777631129254, 'carbonate': 0.18702801899672405, 'chloride': 0.27881796952697535, 'iron_oxide': 0.35502919231069896, 'oxalate': 0.006768214729286988, 'oxychlorine': 0.2533807086878552, 'phyllosilicate': 0.3651507224370013, 'silicate': 0.2933434388817902, 'sulfate': 0.292882412195418, 'sulfide': 0.11754173393741038}


## XGBoost - duration

In [36]:
train_cv_loss_XGB_dur, train_full_clf_XGB_dur, submission_XGB_dur = training.train_tbl(
    df_train=fts_dur_maxabund_ion,
    df_labels=train_labels,
    target_list=target_labels_list,
    df_test=fts_dur_maxabund_ion_VT,
    model_algo='XGB',
    sub_name='XGB_dur'
    )
models_log_loss['XGB_dur'] = models_log_loss.index.map(train_cv_loss_XGB_dur)

/Users/itacdonev/opt/miniconda3/envs/nasamars/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Users/itacdonev/opt/miniconda3/envs/nasamars/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Users/itacdonev/opt/miniconda3/envs/nasamars/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Users/itacdonev/opt/miniconda3/envs/nasamars/lib/python3.9/s


Average Log Loss: 0.2984
Log Loss per Label:
{'basalt': 0.34556434985473283, 'carbonate': 0.3039762745213908, 'chloride': 0.35030911804592496, 'iron_oxide': 0.3612019024268693, 'oxalate': 0.054180436207662774, 'oxychlorine': 0.25880992177133333, 'phyllosilicate': 0.4233393864763624, 'silicate': 0.4181880998372816, 'sulfate': 0.3117083685347533, 'sulfide': 0.15627469664609736}


/Users/itacdonev/opt/miniconda3/envs/nasamars/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


## XGBoost - combo

In [37]:
train_cv_loss_XGB_c_maxa_dur, train_full_clf_XGB_c_maxa_dur, submission_XGB_c_maxa_dur = training.train_tbl(
    df_train=combo_maxabund_dur,
    df_labels=train_labels,
    target_list=target_labels_list,
    df_test=combo_maxabund_dur_VT,
    model_algo='XGB',
    sub_name='XGB_c_maxa_dur'
    )
models_log_loss['XGB_c_maxa_dur'] = models_log_loss.index.map(train_cv_loss_XGB_c_maxa_dur)

/Users/itacdonev/opt/miniconda3/envs/nasamars/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Users/itacdonev/opt/miniconda3/envs/nasamars/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Users/itacdonev/opt/miniconda3/envs/nasamars/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Users/itacdonev/opt/miniconda3/envs/nasamars/lib/python3.9/s


Average Log Loss: 0.2088
Log Loss per Label:
{'basalt': 0.25283269836521766, 'carbonate': 0.1568007914775074, 'chloride': 0.2324278729797448, 'iron_oxide': 0.2708302561882614, 'oxalate': 0.026917641104946738, 'oxychlorine': 0.19666697024059704, 'phyllosilicate': 0.30996859815454475, 'silicate': 0.30714183883599977, 'sulfate': 0.25002698590957995, 'sulfide': 0.08430427635356989}


## XGBoost - Ion peaks

In [38]:
train_cv_loss_XGB_peaks, train_full_clf_XGB_peaks, submission_XGB_peaks = training.train_tbl(
    df_train=fts_ion_peaks,
    df_labels=train_labels,
    target_list=target_labels_list,
    df_test=fts_ion_peaks_VT,
    model_algo='XGB',
    sub_name='XGB_peaks'
    )
models_log_loss['XGB_peaks'] = models_log_loss.index.map(train_cv_loss_XGB_peaks)

/Users/itacdonev/opt/miniconda3/envs/nasamars/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Users/itacdonev/opt/miniconda3/envs/nasamars/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Users/itacdonev/opt/miniconda3/envs/nasamars/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Users/itacdonev/opt/miniconda3/envs/nasamars/lib/python3.9/s


Average Log Loss: 0.2168
Log Loss per Label:
{'basalt': 0.2662005066700371, 'carbonate': 0.15578550758446205, 'chloride': 0.2279696334350056, 'iron_oxide': 0.28814666525320687, 'oxalate': 0.02202209493447068, 'oxychlorine': 0.20259409291290642, 'phyllosilicate': 0.3358335958875466, 'silicate': 0.2946318480355391, 'sulfate': 0.2899387708749964, 'sulfide': 0.08440535310957843}


/Users/itacdonev/opt/miniconda3/envs/nasamars/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


## XGBoost - Temp bin + Ion peaks

In [39]:
train_cv_loss_XGB_tempb_peaks, train_full_clf_XGB_tempb_peaks, submission_XGB_tempb_peaks = training.train_tbl(
    df_train=combo_maxabund_peaks,
    df_labels=train_labels,
    target_list=target_labels_list,
    df_test=combo_maxabund_peaks_VT,
    model_algo='XGB',
    sub_name='XGB_tempb_peaks'
    )
models_log_loss['XGB_tempb_peaks'] = models_log_loss.index.map(train_cv_loss_XGB_tempb_peaks)

/Users/itacdonev/opt/miniconda3/envs/nasamars/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Users/itacdonev/opt/miniconda3/envs/nasamars/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Users/itacdonev/opt/miniconda3/envs/nasamars/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Users/itacdonev/opt/miniconda3/envs/nasamars/lib/python3.9/s


Average Log Loss: 0.1839
Log Loss per Label:
{'basalt': 0.22668344263328524, 'carbonate': 0.13014411018843777, 'chloride': 0.2109755146663402, 'iron_oxide': 0.25353649973288445, 'oxalate': 0.01147664093265209, 'oxychlorine': 0.17701662963923498, 'phyllosilicate': 0.2564108020572279, 'silicate': 0.26046270310180664, 'sulfate': 0.23697255192141126, 'sulfide': 0.0754287604499344}


## XGBoost opt - Temp bin + Ion peaks

In [19]:
train_cv_loss_XGB_tempb_peaks_opt, train_full_clf_XGB_tempb_peaks_opt, submission_XGB_tempb_peaks_opt = training.train_tbl(
    df_train=combo_maxabund_peaks,
    df_labels=train_labels,
    target_list=target_labels_list,
    df_test=combo_maxabund_peaks_VT,
    model_algo='XGB_opt',
    sub_name='XGB_tempb_peaks_opt'
    )
models_log_loss['XGB_tempb_peaks_opt'] = models_log_loss.index.map(train_cv_loss_XGB_tempb_peaks_opt)

/Users/itacdonev/opt/miniconda3/envs/nasamars/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Users/itacdonev/opt/miniconda3/envs/nasamars/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Users/itacdonev/opt/miniconda3/envs/nasamars/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Users/itacdonev/opt/miniconda3/envs/nasamars/lib/python3.9/s


Average Log Loss: 0.1707
Log Loss per Label:
{'basalt': 0.19504383100727152, 'carbonate': 0.12725561255970502, 'chloride': 0.20048671679044144, 'iron_oxide': 0.24008311371277063, 'oxalate': 0.012315083744781958, 'oxychlorine': 0.164854814627361, 'phyllosilicate': 0.2399570535617075, 'silicate': 0.23931971397608573, 'sulfate': 0.21280561231900258, 'sulfide': 0.07529303845939923}


In [20]:
submission_XGB_tempb_peaks_opt.head()

,basalt,carbonate,chloride,iron_oxide,oxalate,oxychlorine,phyllosilicate,silicate,sulfate,sulfide
sample_id,,,,,,,,,,
S0766,0.002123,0.004970,0.003253,0.004333,0.000863,0.003197,0.006514,0.799322,0.003398,0.001778
S0767,0.024597,0.007265,0.000920,0.015660,0.000863,0.005500,0.004605,0.121936,0.022871,0.000727
S0768,0.434360,0.072563,0.000853,0.006407,0.001689,0.003432,0.135144,0.939239,0.030835,0.000634
S0769,0.004025,0.003047,0.005161,0.010979,0.000863,0.993456,0.011854,0.006675,0.065841,0.002423
S0770,0.004413,0.001155,0.003610,0.195763,0.001070,0.996430,0.976062,0.000976,0.004452,0.001012


## XGBopt - Temp bin + Ion peaks + slope_tt

In [55]:
train_cv_loss_XGB_tempb_peaks_slope_opt, train_full_clf_XGB_tempb_peaks_slope_opt, submission_XGB_tempb_peaks_slope_opt =\
    training.train_tbl(
        df_train=combo_maxabund_peaks_slope,
        df_labels=train_labels,
        target_list=target_labels_list,
        df_test=combo_maxabund_peaks_slope_VT,
        model_algo='XGB_opt',
        sub_name='XGB_tempb_peaks_slope_opt'
    )
models_log_loss['XGB_tempb_peaks_slope_opt'] = models_log_loss.index.map(train_cv_loss_XGB_tempb_peaks_slope_opt)

/Users/itacdonev/opt/miniconda3/envs/nasamars/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Users/itacdonev/opt/miniconda3/envs/nasamars/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Users/itacdonev/opt/miniconda3/envs/nasamars/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Users/itacdonev/opt/miniconda3/envs/nasamars/lib/python3.9/s


Average Log Loss: 0.1686
Log Loss per Label:
{'basalt': 0.19520568177743663, 'carbonate': 0.12031231878582822, 'chloride': 0.1963325934969151, 'iron_oxide': 0.24169911216776438, 'oxalate': 0.012315083744781958, 'oxychlorine': 0.16530498244485917, 'phyllosilicate': 0.23245570364568774, 'silicate': 0.2323883510399649, 'sulfate': 0.21482440203037304, 'sulfide': 0.07531181614802744}


***

## Model Summary

- Can we do different models per different target?

In [56]:
models_log_loss.style.highlight_min(axis=1, props='color:darkblue; background-color:lightblue;')

,LR_reg,XGB_opt,XGB_tempb_peaks_opt,XGB_tempb_peaks_slope_opt
basalt,0.276783,0.196651,0.195044,0.195206
carbonate,0.261255,0.121020,0.127256,0.120312
chloride,0.284298,0.204338,0.200487,0.196333
iron_oxide,0.382470,0.226116,0.240083,0.241699
oxalate,0.004059,0.029273,0.012315,0.012315
oxychlorine,0.299150,0.194202,0.164855,0.165305
phyllosilicate,0.407476,0.259516,0.239957,0.232456
silicate,0.351685,0.233179,0.239320,0.232388
sulfate,0.344719,0.203174,0.212806,0.214824
sulfide,0.147136,0.078807,0.075293,0.075312


In [57]:
models_log_loss.agg('mean').sort_values()

XGB_tempb_peaks_slope_opt    0.168615
XGB_tempb_peaks_opt          0.170741
XGB_opt                      0.174628
LR_reg                       0.275903
dtype: float64

# Predict Validation on Individual Labels

Compute predictions for each label depending on which model performed the best for that label. Validate the results on the validation sample only since we have the labels.

In [60]:
submission_by_label = pd.read_csv(config.DATA_DIR + 'submission_format.csv', 
                             index_col='sample_id')
submission_by_label = submission_by_label
print(submission_by_label.shape)

log_loss_ind_label = {}

for label in target_labels_list:
    if label == 'oxalate':
        submission_by_label[label] = submission_LR[label]
    elif label in ['carbonate', 'iron_oxide', 'silicate', 'sulfate']:
        submission_by_label[label] = submission_XGB_opt[label]
    else:
        submission_by_label[label] = submission_XGB_tempb_peaks_opt[label]
    
    # Compute log-loss
    ll = log_loss(valid_labels[label], submission_by_label.iloc[:valid_labels.shape[0]][label])
    log_loss_ind_label[label] = ll

models_log_loss['Ind_labels'] = models_log_loss.index.map(log_loss_ind_label)

submission_by_label.to_csv(config.MODELS_DIR + 'ind_label' + '.csv')

print(f'Average Log Loss: {np.mean(list(log_loss_ind_label.values()))}')
log_loss_ind_label

(804, 10)
Average Log Loss: 0.15643182801666303


{'basalt': 0.13031078681986263,
 'carbonate': 0.0973891174027763,
 'chloride': 0.18781681326631164,
 'iron_oxide': 0.30508353391387294,
 'oxalate': 0.007415226945737933,
 'oxychlorine': 0.16011232950640428,
 'phyllosilicate': 0.24001772800752888,
 'silicate': 0.15213770120553258,
 'sulfate': 0.21562710335758695,
 'sulfide': 0.06840793974101607}

- Adding slope model does not improve CV - 0.16005

In [119]:
submission_by_label.tail()

,basalt,carbonate,chloride,iron_oxide,oxalate,oxychlorine,phyllosilicate,silicate,sulfate,sulfide
sample_id,,,,,,,,,,
S1565,0.046487,0.337855,0.034070,0.108605,0.069877,0.035157,0.007689,0.140228,0.008094,0.021103
S1566,0.011090,0.067778,0.004769,0.004846,0.000071,0.017374,0.087756,0.005926,0.083946,0.011372
S1567,0.024199,0.167448,0.010104,0.001514,0.000390,0.005611,0.020656,0.003359,0.021393,0.002589
S1568,0.025338,0.024485,0.010097,0.002279,0.001032,0.254403,0.363807,0.003484,0.006630,0.113221
S1569,0.060904,0.337855,0.024309,0.124953,0.004330,0.041716,0.012045,0.169667,0.007888,0.025694


In [110]:
models_log_loss.style.highlight_min(axis=1, props='color:darkblue; background-color:lightblue;')

,LR_reg,XGB,XGB_opt,SVC,XGB_dur,XGB_c_maxa_dur,XGB_peaks,XGB_tempb_peaks,XGB_tempb_peaks_opt,Ind_labels
basalt,0.276783,0.225651,0.196651,0.250708,0.345564,0.252833,0.266201,0.226683,0.195044,0.130311
carbonate,0.261255,0.132256,0.121020,0.187028,0.303976,0.156801,0.155786,0.130144,0.127256,0.097389
chloride,0.284298,0.227206,0.204338,0.278818,0.350309,0.232428,0.227970,0.210976,0.200487,0.187817
iron_oxide,0.382470,0.254578,0.226116,0.355029,0.361202,0.270830,0.288147,0.253536,0.240083,0.305084
oxalate,0.004059,0.027906,0.029273,0.006768,0.054180,0.026918,0.022022,0.011477,0.012315,0.007415
oxychlorine,0.299150,0.208509,0.194202,0.253381,0.258810,0.196667,0.202594,0.177017,0.164855,0.160112
phyllosilicate,0.407476,0.297309,0.259516,0.365151,0.423339,0.309969,0.335834,0.256411,0.239957,0.240018
silicate,0.351685,0.257565,0.233179,0.293343,0.418188,0.307142,0.294632,0.260463,0.239320,0.152138
sulfate,0.344719,0.224597,0.203174,0.292882,0.311708,0.250027,0.289939,0.236973,0.212806,0.215627
sulfide,0.147136,0.078318,0.078807,0.117542,0.156275,0.084304,0.084405,0.075429,0.075293,0.068408


# Test SAM-testbed samples

In [22]:
# Select only SAM-testbed samples
tr_sam = metadata[(metadata.split.isin(['train', 'val'])) & (metadata.instrument_type == 'sam_testbed')].copy()
print(f'Number of SAM samples: {tr_sam.shape}')

# Get the index and sample_id
tr_sam_ids = tr_sam['sample_id']
tr_sam_idx = tr_sam.index

Number of SAM samples: (12, 5)


In [44]:
tr_sam_y = train_labels[train_labels.sample_id.isin(tr_sam_ids)]
log_loss_sam = {}
sam_preds = pd.DataFrame(index=tr_sam_idx)

for label in target_labels_list:
    #print(colored(label, 'blue'))
    if label == 'oxalate':
        df = fts_maxrelabund_tempion[fts_maxrelabund_tempion.index.isin(tr_sam_ids)]
        assert df.shape[0] == tr_sam_y.shape[0]
        clf = train_full_clf_LR[label]
    elif label in ['carbonate', 'iron_oxide', 'silicate', 'sulfate']:
        df = fts_maxrelabund_tempion[fts_maxrelabund_tempion.index.isin(tr_sam_ids)]
        assert df.shape[0] == tr_sam_y.shape[0]
        clf = train_full_clf_XGB_opt[label]
    else:
        df = combo_maxabund_peaks[combo_maxabund_peaks.index.isin(tr_sam_ids)]
        assert df.shape[0] == tr_sam_y.shape[0]
        clf = train_full_clf_XGB_tempb_peaks_opt[label]
    
    # Compute predictions
    preds = clf.predict_proba(df)[:,1]
    sam_preds[label] = preds
    
    # Compute log-loss per label
    y_true = tr_sam_y[label]
    ll = log_loss(y_true, preds, labels=(0,1))
    #print(colored(f'Log-Loss: {label} = {ll}', 'blue'))
    log_loss_sam[label] = ll

/Users/itacdonev/opt/miniconda3/envs/nasamars/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Users/itacdonev/opt/miniconda3/envs/nasamars/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Users/itacdonev/opt/miniconda3/envs/nasamars/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Users/itacdonev/opt/miniconda3/envs/nasamars/lib/python3.9/s

In [52]:
print(colored(f'Avg log-loss: {np.mean(list(log_loss_sam.values()))}', 'blue'))
log_loss_sam

Avg log-loss: 0.01105341586981636


{'basalt': 0.00817040930269286,
 'carbonate': 0.02323586850737532,
 'chloride': 0.008542213841186216,
 'iron_oxide': 0.003749249726145839,
 'oxalate': 0.0004237058241432469,
 'oxychlorine': 0.01820063612346227,
 'phyllosilicate': 0.019062379258684814,
 'silicate': 0.004288394246638442,
 'sulfate': 0.010433380724862218,
 'sulfide': 0.014427921142972385}

In [51]:
sam_preds

,basalt,carbonate,chloride,iron_oxide,oxalate,oxychlorine,phyllosilicate,silicate,sulfate,sulfide
754,0.006946,0.003540,0.005339,0.001839,0.000026,0.002438,0.011313,0.005466,0.002143,0.002136
755,0.008779,0.920872,0.003459,0.003765,0.000226,0.013575,0.011897,0.004097,0.959072,0.033414
756,0.010334,0.014328,0.004074,0.003082,0.000039,0.009962,0.009295,0.004370,0.005751,0.012877
757,0.004344,0.005396,0.010264,0.005105,0.000520,0.004327,0.006499,0.004662,0.003414,0.004302
758,0.002916,0.060261,0.025475,0.005565,0.000598,0.003688,0.012882,0.003239,0.010757,0.014599
759,0.003384,0.012546,0.011355,0.005148,0.000652,0.002271,0.006722,0.002968,0.009270,0.009117
760,0.027260,0.005765,0.006636,0.003390,0.000507,0.868112,0.036084,0.008661,0.004605,0.046630
761,0.007358,0.005153,0.004288,0.005252,0.000067,0.004363,0.012691,0.003110,0.007553,0.001902
762,0.008387,0.045953,0.009024,0.001510,0.001083,0.024610,0.906038,0.005306,0.004900,0.024585
763,0.004823,0.013377,0.002018,0.004249,0.000460,0.003440,0.005528,0.001632,0.005135,0.008699
